**US Electrical Disturbances (2019-2023)**

**Data source** : https://www.oe.netl.doe.gov/OE417_annual_summary.aspx

###Imports

In [31]:
import pandas as pd
import os
import xlrd

###Ignore Warnings

In [32]:
import warnings
warnings.filterwarnings('ignore')

###Import Data and concatenate data from several sheets into single dataframe

In [33]:
years = list(range(2019,2024,1))

data_frames = []

for year in years:
    file_path=f"/content/Data/{year}_Annual_Summary.xls"
    wb = xlrd.open_workbook(file_path, logfile=open(os.devnull, 'w')) #This line was added to supress xls warnings
    sheet_df = pd.read_excel(wb,header=1)
    sheet_df['Year'] = year
    data_frames.append(sheet_df)


# Concatenate all the DataFrames into one
combined_df = pd.concat(data_frames, ignore_index=True, axis=0)
combined_df.head()

,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,Year
0,January,01/05/2019,13:19:00,01/05/2019,15:07:00,Washington:,WECC,Complete loss of Interpersonal Communication a...,System Operations,0,Unknown,2019
1,January,01/06/2019,03:00:00,01/09/2019,07:00:00,Washington:,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,230,230000,2019
2,January,01/06/2019,17:56:00,01/06/2019,21:52:00,California: Sacramento County;,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,300,90382,2019
3,January,01/06/2019,01:00:00,01/06/2019,12:00:00,"Washington: King County, Thurston County, Pier...",WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,Unknown,230000,2019
4,January,01/07/2019,20:57:00,01/07/2019,21:32:00,Michigan:,RF,Complete loss of monitoring or control capabil...,System Operations,0,0,2019


###Backup the combined_df

In [34]:
saved_df = combined_df.copy()

###Remove Duplicates

In [35]:
print(combined_df.duplicated().sum())
combined_df = combined_df.drop_duplicates()
print(combined_df.duplicated().sum())

17
0


In [36]:
print('Number of rows =',combined_df.shape[0])
print('Number of columns =',combined_df.shape[1])

Number of rows = 1769
Number of columns = 12


##Convert to proper data types and clean the data

In [37]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1769 entries, 0 to 1785
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Month                         1769 non-null   object
 1   Date Event Began              1737 non-null   object
 2   Time Event Began              1769 non-null   object
 3   Date of Restoration           1769 non-null   object
 4   Time of Restoration           1769 non-null   object
 5   Area Affected                 1769 non-null   object
 6   NERC Region                   1760 non-null   object
 7   Alert Criteria                1769 non-null   object
 8   Event Type                    1769 non-null   object
 9   Demand Loss (MW)              1769 non-null   object
 10  Number of Customers Affected  1768 non-null   object
 11  Year                          1769 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 179.7+ KB


###Clean the months colummn

In [38]:
combined_df['Month'].unique()

array(['January ', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December',
       'October '], dtype=object)

In [39]:
combined_df['Month'] = combined_df['Month'].str.strip()
combined_df['Month'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

###Remove 'Unknown' throughout the dataset

In [40]:
combined_df = combined_df.replace('Unknown', pd.NA, regex=True)

###Joining date time into single column

In [41]:
combined_df['Datetime Event Began'] = pd.to_datetime(combined_df['Date Event Began'].astype(str) + ' ' + combined_df['Time Event Began'].astype(str),format='mixed', errors='coerce')
combined_df['Datetime of Restoration'] = pd.to_datetime(combined_df['Date of Restoration'].astype(str) + ' ' + combined_df['Time of Restoration'].astype(str),format='mixed', errors='coerce')

###Handle data entry error in datetime

In [42]:
combined_df[combined_df['Datetime of Restoration']<combined_df['Datetime Event Began']]

,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,Year,Datetime Event Began,Datetime of Restoration
179,August,08/18/2019,16:47:00,08/18/2018,23:00:00,Texas:,TRE,Uncontrolled loss of 300 Megawatts or more of ...,Distribution Interruption,752,86373,2019,2019-08-18 16:47:00,2018-08-18 23:00:00


In [43]:
combined_df['Datetime of Restoration'].loc[(combined_df['Datetime of Restoration']=='2018-08-18 23:00:00') & (combined_df['Year']==2019)] = '2019-08-18 23:00:00'

###Create new column for Event Duration in hours

In [44]:
combined_df['Event Duration (hours)'] = combined_df['Datetime of Restoration'] - combined_df['Datetime Event Began']
combined_df['Event Duration (hours)'] = combined_df['Event Duration (hours)'].dt.total_seconds() / 3600
combined_df['Event Duration (hours)'] = combined_df['Event Duration (hours)'].round().astype('Int64')

###Dropping unwanted columns

In [45]:
#droping unwanted columns
combined_df = combined_df.drop(columns=['Date Event Began','Time Event Began','Date of Restoration','Time of Restoration','NERC Region','Alert Criteria'])

###Correct the Event type column

In [46]:
combined_df['Event Type'].str.replace("- ","").unique()

array(['System Operations', 'Severe Weather', 'Suspicious Activity',
       'Actual Physical Attack', 'Vandalism', 'Transmission Interruption',
       'Fuel Supply Deficiency',
       'Severe Weather/Transmission Interruption',
       'Generation Inadequacy',
       'Severe Weather/Distribution Interruption',
       'Distribution Interruption',
       'Transmission Interruption/Distribution Interruption',
       'Natural Disaster', 'Sabotage', 'Potential Physical Attack',
       'Cyber Event', 'Severe Weather/Transmisison Interruption',
       'Actual Physical Attack/Vandalism', 'Suspicious activity',
       'Sever Weather', 'Weather or natural disaster',
       'Vandalism\xa0Theft',
       'Failure at high voltage substation or switchyard',
       'Transmission equipment failure', <NA>, 'Other',
       'Failure at high voltage substation or switchyard\xa0Other',
       'Physical attack\xa0Vandalism\xa0Suspicious activity',
       'Failure at high voltage substation or switchyard\xa0We

In [47]:
combined_df['Event Type'].str.replace("- ","").str.replace(u"\xa0"," ") .str.replace("/"," ").unique().size

47

In [48]:
mistakes = {
    "- ":"",
    u"\xa0":" ",
    "/":" ",
    "Transmisison":"Transmission",
    "activity":"Activity",
    "event":"Event",
    " or ":" ",
    "physical":"Physical",
    "attack":"Attack",
    "natural disaster":"Natural Disaster",
    "(information technology)" : ''
}

In [49]:
for key,value in mistakes.items():
    combined_df['Event Type'] = combined_df['Event Type'].str.replace(key,value)

In [50]:
combined_df['Event Type'].unique().size

45

###Correct the Area Affected Column

In [51]:
combined_df['Area Affected'].unique()

array(['Washington:', 'California: Sacramento County;',
       'Washington: King County, Thurston County, Pierce County;',
       'Michigan:', 'New Mexico: San Juan County;',
       'Montana: Valley County;', 'Maine:',
       'Missouri: Jackson County; Kansas: Johnson County;',
       'Missouri: Nebraska:', 'New Hampshire:',
       'North Carolina: South Carolina:', 'Pennsylvania:', 'California:',
       'Texas: Nueces County;', 'Nebraska:', 'Colorado: Larimer County;',
       'Ohio:', 'Utah: Iron County;', 'Louisiana: Washington Parish;',
       'Illinois: Scott County;', 'Illinois: Pike County;', 'Nevada',
       'Massachusetts', 'California', 'Arkansas', 'Michigan',
       'Pennsylvania', 'Washington', 'New Jersey', 'Utah', 'Texas',
       'Wisconsin, Minnesota, Iowa, Illinois', 'Virginia',
       'Ohio, Virginia, West Virginia', 'Ohio',
       'Connecticut, Massachusetts, New Hampshire, Maine, Vermont, Rhode Island',
       'Iowa', 'South Carolina', 'Oklahoma, Arkansas, Louisiana',

In [52]:
#getting only state names
combined_df['Area Affected'][combined_df['Area Affected'].str.extract('^([A-Za-z ]+):',expand=False).isna() == False] = combined_df['Area Affected'].str.extract('^([A-Za-z ]+):',expand=False)

In [53]:
combined_df['Area Affected'].unique()

array(['Washington', 'California', 'Michigan', 'New Mexico', 'Montana',
       'Maine', 'Missouri', 'New Hampshire', 'North Carolina',
       'Pennsylvania', 'Texas', 'Nebraska', 'Colorado', 'Ohio', 'Utah',
       'Louisiana', 'Illinois', 'Nevada', 'Massachusetts', 'Arkansas',
       'New Jersey', 'Wisconsin, Minnesota, Iowa, Illinois', 'Virginia',
       'Ohio, Virginia, West Virginia',
       'Connecticut, Massachusetts, New Hampshire, Maine, Vermont, Rhode Island',
       'Iowa', 'South Carolina', 'Oklahoma, Arkansas, Louisiana',
       'Oklahoma', 'Oregon', 'New York', 'Minnesota', 'Mississippi',
       'Arizona', 'Alabama', 'Indiana', 'West Virginia', 'Kentucky',
       'North Dakota', 'Idaho', 'South Dakota', 'Wisconsin',
       'District of Columbia', 'Maryland', 'Wyoming', 'Tennessee',
       'Kansas', 'Connecticut', 'Florida',
       'Northern and Central California;', 'Western NY', 'Delaware',
       'Georgia', 'Central Oklahoma', 'Texas panhandle, SE NM', 'Vermont',
       '

###Correct Demand Loss column

In [54]:
combined_df['Demand Loss (MW)'].unique()

array(['0', '230', '300', <NA>, '11', '133200', '190', '8', '42', '3',
       '182', '740', '50', '58', '5', '537', '116', '150', '7', '29',
       '691', '30', '20', '347', '982', '115', '47', '53', '3189', '26',
       '452', '200', '60', '49', '54', '66', '80', '752', '259', '271',
       '885', '25', '3736', '2400', '3190', '285', '1500', '72', '178',
       '35', '2', '1', '18', '22', '4', '219', '87', '540', '75', '500',
       '97', '91', '19', '675', '40', '165', '6062', '237', '412', '122',
       '448', '552', '375', '14', '33480', '32', '36', '475', '21', '125',
       '3907', '2000', '311', '89', '550', '1400', '950', '218', '560',
       '101', '1120', '1419', '459', '795', '409', '712', '829', '7800',
       '917', '205', '1200', '7500', '48', '581', '573', '100', '208',
       '350', '2900', '986', '8180', '1071', '610', '2859', '265', '236',
       '337', '183', '62', '1218', '64', '253', '93', '132', '286', '298',
       0, '6', '185', '411', '45', '841', '13', '164', 

In [55]:
combined_df['Demand Loss (MW)'] = combined_df['Demand Loss (MW)'].replace('Unknown', pd.NA, regex=True)
combined_df['Demand Loss (MW)'] = pd.to_numeric(combined_df['Demand Loss (MW)'], errors='coerce')
combined_df['Demand Loss (MW)'] = combined_df['Demand Loss (MW)'].astype('Int64')

###Correct Number of customers affected column

In [56]:
combined_df["Number of Customers Affected"].unique()

array([<NA>, '230000', '90382', '0', '2', '112530', '116600', '126700',
       '1', '33200', '3370', '233000', '50940', '121000', '132000',
       '118781', '137216', '157274', '94048', '115000', '72332', '54290',
       '58379', '154124', '66000', '231956', '60467', '75290', '34695',
       '51366', '5830', '54071', '65844', '238015', '61008', '68000',
       '100000', '10961', '70000', '55017', '558000', '30907', '340000',
       '82045', '60000', '50000', '34637', '265000', '16199', '56451',
       '52498', '3000', '57000', '2043', '63000', '55730', '72669',
       '400000', '160000', '45000', '165000', '95600', '25', '54535',
       '13720', '5600', '61318', '86373', '61000', '1397', '95000',
       '103779', '2000', '116000', '172278', '77000', '54000', '69524',
       '50072', '737808', '101683', '972000', '82124', '114402', '53943',
       '8000', '80066', '66325', '131', '93000', '107000', '400', '392',
       '7541', '66475', '68138', '30715', '67864', '3120', '130000',
      

In [57]:
combined_df['Number of Customers Affected'] = combined_df['Number of Customers Affected'].replace('Unknown', pd.NA, regex=True)
combined_df['Number of Customers Affected'] = pd.to_numeric(combined_df['Number of Customers Affected'], errors='coerce')
combined_df['Number of Customers Affected'] = combined_df['Number of Customers Affected'].astype('Int64')

In [58]:
combined_df["Number of Customers Affected"].unique()

<IntegerArray>
[  <NA>, 230000,  90382,      0,      2, 112530, 116600, 126700,      1,
  33200,
 ...
      6,     26, 126197, 230330, 127585, 170316, 730000,  50204,  37850,
  36100]
Length: 484, dtype: Int64

##Save dataframe to csv named `electricity_disturbance_data.csv`

In [59]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1769 entries, 0 to 1785
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Month                         1769 non-null   object        
 1   Area Affected                 1769 non-null   object        
 2   Event Type                    1757 non-null   object        
 3   Demand Loss (MW)              1280 non-null   Int64         
 4   Number of Customers Affected  1549 non-null   Int64         
 5   Year                          1769 non-null   int64         
 6   Datetime Event Began          1737 non-null   datetime64[ns]
 7   Datetime of Restoration       1524 non-null   datetime64[ns]
 8   Event Duration (hours)        1497 non-null   Int64         
dtypes: Int64(3), datetime64[ns](2), int64(1), object(3)
memory usage: 143.4+ KB


In [60]:
combined_df.to_csv('/content/electricity_disturbance_data.csv',index=False)